# Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime
import logging
import pandas as pd
import csv
from sqlalchemy import create_engine
import sqlite3

# Getting economic news data from forexfactory.com

- https://www.forexfactory.com/

In [2]:
forcal = []

def setLogger():
    logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='logs_file',
                    filemode='w')
    console = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

def getEconomicCalendar(startlink,endlink):

    # write to console current status
    logging.info("Scraping data for link: {}".format(startlink))

    # get the page and make the soup
    baseURL = "https://www.forexfactory.com/"
    r = requests.get(baseURL + startlink)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")

    # get and parse table data, ignoring details and graph
    table = soup.find("table", class_="calendar__table")

    # do not use the ".calendar__row--grey" css selector (reserved for historical data)
    trs = table.select("tr.calendar__row.calendar_row")
    fields = ["date","time","currency","impact","event","actual","forecast","previous"]

    # some rows do not have a date (cells merged)
    curr_year = startlink[-4:]
    curr_date = ""
    curr_time = ""
    for tr in trs:
        dict = {}

        # fields may mess up sometimes, see Tue Sep 25 2:45AM French Consumer Spending
        # in that case we append to errors.csv the date time where the error is
        try:
            for field in fields:
                data = tr.select("td.calendar__cell.calendar__{}.{}".format(field,field))[0]
                # print(data)
                if field=="date" and data.text.strip()!="":
                    curr_date = data.text.strip()
                elif field=="time" and data.text.strip()!="":
                    # time is sometimes "All Day" or "Day X" (eg. WEF Annual Meetings)
                    if data.text.strip().find("Day")!=-1:
                        curr_time = "12:00am"
                    else:
                        curr_time = data.text.strip()
                elif field=="currency":
                    currency = data.text.strip()
                elif field=="impact":
                    # when impact says "Non-Economic" on mouseover, the relevant
                    # class name is "Holiday", thus we do not use the classname
                    impact = data.find("span")["title"]
                elif field=="event":
                    event = data.text.strip()
                elif field=="actual":
                    actual = data.text.strip()
                elif field=="forecast":
                    forecast = data.text.strip()
                elif field=="previous":
                    previous = data.text.strip()

            date = datetime.datetime.strptime(",".join([curr_year,curr_date,curr_time]),"%Y,%a%b %d,%I:%M%p")
            # date = datetime.datetime.strptime(",".join([curr_year,curr_date,curr_time]),"%Y,%a%b %d,%I:%M%p")
            # date = datetime.datetime.strptime(",".join([curr_year,curr_date]),"%Y,%a%b")
            # time = datetime.datetime.strptime(curr_time, "%d,%I:%M%p")

            dict["Date"] = date.strftime("%Y-%m-%d %H:%M:%S")
            dict["Currency"] = currency
            dict["Impact"] = impact
            dict["Event"] = event
            dict["Actual"] = actual
            dict["Forecast"] = forecast
            dict["Previous"] = previous


            forcal.append(dict)
            # forcal.append(",".join([str(dt),currency,impact,event,actual,forecast,previous]))


        except:
            with open("errors.csv","a") as f:
                csv.writer(f).writerow([curr_year,curr_date,curr_time])

    # exit recursion when last available link has reached
    if startlink==endlink:
        logging.info("Successfully retrieved data")
        return

    # get the link for the next week and follow
    follow = soup.select("a.calendar__pagination.calendar__pagination--next.next")
    follow = follow[0]["href"]
    getEconomicCalendar(follow,endlink)

Original idea
 - https://gist.github.com/pohzipohzi/ad7942fc5545675022c1f31123e64c0c

# Initialize function

In [3]:
setLogger()
getEconomicCalendar("calendar?day=apr24.2020","calendar?day=apr24.2020")

2020-04-24 20:07:46,531 - INFO - Scraping data for link: calendar?day=apr24.2020
2020-04-24 20:07:47,438 - INFO - Successfully retrieved data


# Creating dataframe

In [4]:
df = pd.DataFrame(forcal)
df

,Date,Currency,Impact,Event,Actual,Forecast,Previous
0,2020-04-24 00:30:00,JPY,Low Impact Expected,All Industries Activity m/m,-0.6%,-0.4%,0.6%
1,2020-04-24 02:00:00,GBP,High Impact Expected,Retail Sales m/m,-5.1%,-4.5%,-0.3%
2,2020-04-24 04:00:00,EUR,High Impact Expected,German ifo Business Climate,74.3,79.8,85.9
3,2020-04-24 08:30:00,USD,High Impact Expected,Core Durable Goods Orders m/m,-0.2%,-6.1%,-0.7%
4,2020-04-24 08:30:00,USD,Medium Impact Expected,Durable Goods Orders m/m,-14.4%,-12.0%,1.1%
5,2020-04-24 08:57:00,EUR,Low Impact Expected,Belgian NBB Business Climate,-36.1,-21.0,-10.9
6,2020-04-24 10:00:00,USD,Low Impact Expected,Revised UoM Consumer Sentiment,71.8,67.8,71.0
7,2020-04-24 10:00:00,USD,Low Impact Expected,Revised UoM Inflation Expectations,2.1%,,2.1%


In [5]:
import numpy as np

# Creating surprises column

- Dropping missing values rows

In [6]:
df['Forecast'].replace('', np.nan, inplace=True)
df.dropna(subset=['Forecast'], inplace=True)
df

,Date,Currency,Impact,Event,Actual,Forecast,Previous
0,2020-04-24 00:30:00,JPY,Low Impact Expected,All Industries Activity m/m,-0.6%,-0.4%,0.6%
1,2020-04-24 02:00:00,GBP,High Impact Expected,Retail Sales m/m,-5.1%,-4.5%,-0.3%
2,2020-04-24 04:00:00,EUR,High Impact Expected,German ifo Business Climate,74.3,79.8,85.9
3,2020-04-24 08:30:00,USD,High Impact Expected,Core Durable Goods Orders m/m,-0.2%,-6.1%,-0.7%
4,2020-04-24 08:30:00,USD,Medium Impact Expected,Durable Goods Orders m/m,-14.4%,-12.0%,1.1%
5,2020-04-24 08:57:00,EUR,Low Impact Expected,Belgian NBB Business Climate,-36.1,-21.0,-10.9
6,2020-04-24 10:00:00,USD,Low Impact Expected,Revised UoM Consumer Sentiment,71.8,67.8,71.0


- Checking data types

In [7]:
df.dtypes

Date        object
Currency    object
Impact      object
Event       object
Actual      object
Forecast    object
Previous    object
dtype: object

- Converting from object to float

In [8]:
df['Forecast'] = df.Forecast.str.replace('%', '').astype(float)

In [9]:
df['Actual'] = df.Actual.str.replace('%', '').astype(float)
# df['Actual'] = df['Actual'].astype(str).astype(float)

In [10]:
df.dtypes

Date         object
Currency     object
Impact       object
Event        object
Actual      float64
Forecast    float64
Previous     object
dtype: object

In [11]:
df["Surprises"] = df["Actual"] - df["Forecast"]
df

2020-04-24 20:07:47,540 - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-04-24 20:07:47,541 - INFO - NumExpr defaulting to 8 threads.


,Date,Currency,Impact,Event,Actual,Forecast,Previous,Surprises
0,2020-04-24 00:30:00,JPY,Low Impact Expected,All Industries Activity m/m,-0.6,-0.4,0.6%,-0.2
1,2020-04-24 02:00:00,GBP,High Impact Expected,Retail Sales m/m,-5.1,-4.5,-0.3%,-0.6
2,2020-04-24 04:00:00,EUR,High Impact Expected,German ifo Business Climate,74.3,79.8,85.9,-5.5
3,2020-04-24 08:30:00,USD,High Impact Expected,Core Durable Goods Orders m/m,-0.2,-6.1,-0.7%,5.9
4,2020-04-24 08:30:00,USD,Medium Impact Expected,Durable Goods Orders m/m,-14.4,-12.0,1.1%,-2.4
5,2020-04-24 08:57:00,EUR,Low Impact Expected,Belgian NBB Business Climate,-36.1,-21.0,-10.9,-15.1
6,2020-04-24 10:00:00,USD,Low Impact Expected,Revised UoM Consumer Sentiment,71.8,67.8,71.0,4.0


# Next step import stock index prices...